In [1]:
!pip install bitermplus
!pip install tmplot

In [2]:
import bitermplus as btm
import numpy as np
import pandas as pd
import tmplot as tmp
import plotly.express as px

In [3]:
# IMPORTING DATA
df = pd.read_csv('CR_dataset.csv')
df['report_lemmatized_filtered'] = df['report_lemmatized_filtered'].map(lambda x: " ".join(eval(x)))

In [4]:
df['report_lemmatized_filtered'] = df['report_lemmatized_filtered'].astype(str)

In [5]:
texts = df['report_lemmatized_filtered'].tolist()
#.str.strip().tolist()

In [6]:
# PREPROCESSING
# Obtaining terms frequency in a sparse matrix and corpus vocabulary
X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
tf = np.array(X.sum(axis=0)).ravel()

# Vectorizing documents
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))

# Generating biterms
biterms = btm.get_biterms(docs_vec)

In [29]:
biterms

KeyboardInterrupt: 

In [7]:
stop

NameError: name 'stop' is not defined

## Loop

In [ ]:
# Renyi entropy can be used to estimate the optimal number of topics: 
# just fit several models with a different number of topics and choose the number of topics 
# for which the Renyi entropy is the least.

# 1. Loop

n_topics = range(4,40)

# lists to store the metrics
perplexity_list = []
coherence_list = []
entropy_list = []

# looping through n_topics
for t in n_topics:
    model = btm.BTM(X, vocabulary, seed=1, T=t, alpha=50/8, beta=0.01, M=20) # alpha=50/8 -> default is =1
    model.fit(biterms, iterations=50) # iterations=600 is the default, but this takes forever.
    p_zd = model.transform(docs_vec)
    
    #perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 8) # idk what X and 8 are
    #coherence = btm.coherence(model.matrix_topics_words_, X, M=20) # idk what X is
    
    perplexity = model.perplexity_
    perplexity_list.append(perplexity)
    
    coherence = model.coherence_
    coherence_list.append(coherence)
    
    entropy = btm.entropy(model.matrix_topics_words_)
    entropy_list.append(entropy)

In [ ]:
coherence_means = []

for lists in coherence_list:
    mean = lists.mean()
    coherence_means.append(mean)

In [ ]:
zipped = list(zip(n_topics, entropy_list, perplexity_list, coherence_means))

new_df = pd.DataFrame(zipped, columns=['T','Entropy','Perplexity','Average_coherence_score'])

In [ ]:
px.line(new_df, x='T', y='Entropy', template='plotly_white', width=650, height=400,color_discrete_sequence=['#533A71','#9DD9D2'])
# T = 6 has the lowest entropy

In [ ]:
px.line(new_df, x='T', y=['Perplexity','Average_coherence_score'], template="plotly_white",
       width=650)

In [ ]:
# normalizing perplexity and avg coherence score
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X1 = new_df[['Perplexity']]
X2 = new_df[['Average_coherence_score']]

new_df['Perplexity_norm'] = min_max_scaler.fit_transform(X1)
new_df['Coherence_norm'] = min_max_scaler.fit_transform(X2)

In [ ]:
xx = new_df.rename(columns={'Perplexity_norm':'Perplexity (normalized)','Coherence_norm':'Average coherence scores (normalized)'})
px.line(xx, x='T', y=['Perplexity (normalized)','Average coherence scores (normalized)'], template="plotly_white",
       color_discrete_sequence=['#533A71','#9DD9D2'], labels={'Perplexity_norm':'Perplexity (normalized)',
                                                             'Coherence_norm':'Average coherence scores (normalized)',
                                                             'variable':'metric',
                                                             'T':'Number of topics'})

# so 11 topics, or 21-24-28?

In [ ]:
# Perplexity needs to be low
# Coherence needs to be high

-----------

## Final model(s)

#### 28 topics

In [8]:
model = btm.BTM(X, vocabulary, seed=1, T=28, M=20, alpha=50/8, beta=0.01) # seed 12321
model.fit(biterms, iterations=50) #=20
p_zd = model.transform(docs_vec)

perplexity = model.perplexity_
coherence = model.coherence_

print(perplexity)
print(coherence.mean())

100%|██████████| 1096/1096 [00:24<00:00, 45.28it/s] 


1524.4031358804264
-25.720860581198476


In [9]:
tmp.report(model=model, docs=texts, width=200, height=500)

In [16]:
doc_topics = btm.get_docs_top_topic(texts, model.matrix_docs_topics_)

In [19]:
doc_topics['topic1'] = doc_topics['label']+1

In [20]:
doc_topics.topic1.value_counts()

14    297
1     212
20    114
27     70
11     60
19     45
7      41
2      34
25     33
16     32
22     27
8      26
3      23
10     13
21      9
24      9
13      8
18      8
5       8
6       7
26      7
12      4
17      4
4       3
23      2
Name: topic1, dtype: int64

In [23]:
doct = doc_topics['topic1'].value_counts()
doct = pd.DataFrame(doct).reset_index().rename(columns={'index':'Topic','topic1':'Number of documents'})

In [24]:
fig=px.bar(doct, x='Topic', y='Number of documents', template='plotly_white', labels={'label_1-28':'Topic'},
       color_discrete_sequence=['#533A71'])
fig.update_xaxes(nticks=30)
fig.update_layout(bargap=0.02)
fig.show()

----------

#### 13 topics

In [ ]:
model = btm.BTM(X, vocabulary, seed=12321, T=13, M=20, alpha=50/8, beta=0.01)
model.fit(biterms, iterations=50) #=20
p_zd = model.transform(docs_vec)

perplexity = model.perplexity_
coherence = model.coherence_

print(perplexity)
print(coherence.mean())

In [ ]:
tmp.report(model=model, docs=texts, width=200, height=500)

In [ ]:
doc_topics_2 = btm.get_docs_top_topic(texts, model.matrix_docs_topics_)
doc_topics_2['label_1-28'] = doc_topics_2['label']+1

In [ ]:
fig=px.histogram(doc_topics_2, x='label_1-28', template='plotly_white', labels={'label_1-28':'Topic'},
       color_discrete_sequence=['#533A71'])
fig.update_xaxes(nticks=14)
fig.show()